# Simulation #2
Having written the functions required to implement estimation of $L$, we now experiment with a few different model parameters. First, however, it will be helpful to wrap all of the helper functions into a single ```estimate_L``` function.

In [1]:
using JLD

In [14]:
include("functions.jl");

In [3]:
?estimate_L

search: estimate_L estimate SkewSymmetricMatrixSpace SkewSymmetricMatrixShape



```
estimate_L(state_space, sample_size, S, φ, ϵ, basis_grid_size, integral_resolution; dist)
```

Combines all of the helper functions into a single API for estimating the matrix L.

Returns L.


In [4]:
state_space = Dict("min"=>0, "max"=>2π);

## Varying the integral resolution
As an initial test, I don't expect that varying the resolution of the integral calculations should have a significant effect on the calculation of $L$.

In [ ]:
integral_resolutions = [100, 200, 500, 1000];
results = [];
for res in integral_resolutions
    L = estimate_L(state_space, 5000, S, φ, 0.57, 40, res);
    push!(results, L);
    println("Estimation complete for: $res")
end

Academic license - for non-commercial use only - expires 2021-08-05
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1600 rows, 6600 columns and 6192871 nonzeros
Model fingerprint: 0x8c086894
Model has 6600 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e-13, 1e+00]
  Objective range  [5e-04, 5e-04]
  QObjective range [2e-02, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 2.07s
Presolved: 1600 rows, 6600 columns, 6192871 nonzeros
Presolved model has 6600 quadratic objective terms
Ordering time: 0.02s

Barrier statistics:
 AA' NZ     : 1.279e+06
 Factor NZ  : 1.281e+06 (roughly 14 MBytes of memory)
 Factor Ops : 1.367e+09 (roughly 2 seconds per iteration)
 Threads    : 2

                  Objective                Residual
Iter       Primal  

In [ ]:
d = Dict(
    "results"=>results, 
    "integral_resolutions"=>integral_resolutions,
);

In [ ]:
save("estimations/experiment_integral_resolutions.jld", "d", d)

In [ ]:
for i in 1:4
    resolution = integral_resolutions[i];
    λ, Λ = ordered_eigendecomp(L);
    println("With integral resolution $resolution, leading eigenvalue is $λ[1]")
end

## Varying $\epsilon$
Varying the bandwidth of the basis functions however should have an effect, since this will by definition alter the structure of the subspace in which we are estimating $\mathcal{L}$.

In [ ]:
epsilons = [0.01, 0.1, 0.5, 1, 10];
results = [];
for epsilon in epsilons
    L = estimate_L(state_space, 5000, S, φ, epsilon, 40, 100);
    push!(results, L);
    println("Estimation complete for: $epsilon")
end

In [ ]:
d = Dict(
    "results"=>results, 
    "epsilons"=>epsilons,
);

In [ ]:
save("estimations/experiment_epsilons.jld", "d", d)

In [ ]:
for i in 1:4
    sample_size = sample_sizes[i];
    on = opnorm(results[i]);
    println("With sample size $sample_size, opnorm is $on")
end

Clearly the results show that...

## Varying the dynamical map
I will try three different maps:
1. The standard map. This is known to have Lebesgue measure as the invariant density.
2. The cat map. This too should have a constant function as its invariant density.
3. The perturbed cat map. Even by perturbing the cat map only slightly, we should here obtain a non-constant function as the invariant density.